In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import sys
if not '..' in sys.path:
    sys.path.append('..')
from Pipeline.PPP import *

In [3]:
PATH_PIPE = '../Pipeline/'
PATH_RAW = '../data/raw_data/'
PATH_DATA = '../data/data/'
PATH_MODELS = '../../models/'
PATH_IMAGES = '../assets/images/'

## Load data

Either with separate train and test files or...

In [4]:
# New data:
test_socc = pd.read_csv(PATH_DATA + "socc.csv")

#X_test_socc = ['text', 'sentiment', 'readability', 'num tokens', 'num char', 'lemmas', 'pos tags']
#y_test_socc = 

In [6]:
test_socc

,article_id,comment_counter,title,globe_url,url,comment_text,is_constructive,is_constructive:confidence,toxicity_level,toxicity_level:confidence,did_you_read_the_article,did_you_read_the_article:confidence,annotator_comments,expert_is_constructive,expert_toxicity_level,expert_comments
0,23462276,source1_23462276_4,Apple Watch: It's the precise opposite of a la...,http://www.theglobeandmail.com/opinion/as-tech...,http://www.sfu.ca/content/dam/sfu/discourse-la...,"While technology does march on, sometimes it t...",yes,1.0000,1,1,1,1.0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n,NaN,NaN,NaN
1,23462276,source1_23462276_10,Apple Watch: It's the precise opposite of a la...,http://www.theglobeandmail.com/opinion/as-tech...,http://www.sfu.ca/content/dam/sfu/discourse-la...,Everyone is still missing the point of what th...,yes,1.0000,1\n2,0.9111\n0.0889,1,1.0,\n\n\n\n\n\n\n\n\n\n,NaN,NaN,NaN
2,23462276,source1_23462276_11_1,Apple Watch: It's the precise opposite of a la...,http://www.theglobeandmail.com/opinion/as-tech...,http://www.sfu.ca/content/dam/sfu/discourse-la...,You've never used an Apple product have you?,no,1.0000,1\n2,0.6308\n0.2937,1,1.0,\n\n\n\n\n\n\n\n\n\n\n\n\n,NaN,NaN,NaN
3,23462276,source1_23462276_25,Apple Watch: It's the precise opposite of a la...,http://www.theglobeandmail.com/opinion/as-tech...,http://www.sfu.ca/content/dam/sfu/discourse-la...,You may be using a blackberry. I'm still using...,no,1.0000,1\n2,0.7346\n0.2023,1,1.0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n,NaN,NaN,NaN
4,26373964,source1_26373964_63,Don't be fooled by the (surprise!) budget surplus,http://www.theglobeandmail.com/opinion/dont-be...,http://www.sfu.ca/content/dam/sfu/discourse-la...,"Of course we all know, unfortunately, there is...",no,0.5963,2\n1,0.4429\n0.3066,1,1.0,\n\n\n\n\n\n,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,32803341,source1_32803341_121,"Thank you, Hillary. Now women know retreat is ...",http://www.theglobeandmail.com/opinion/thank-y...,http://www.sfu.ca/content/dam/sfu/discourse-la...,Hillary Clinton wasn't defeated for being a wo...,yes,0.6710,2\n1,0.671\n0.329,1,1.0,\n\n,yes,1.0,I wish the Globe and Mail used spellcheck befo...
1039,32803341,source1_32803341_122,"Thank you, Hillary. Now women know retreat is ...",http://www.theglobeandmail.com/opinion/thank-y...,http://www.sfu.ca/content/dam/sfu/discourse-la...,Very well said... and thank you for shouting i...,yes,1.0000,2\n1,0.671\n0.329,1,1.0,\n\n,yes,1.0,NaN
1040,32803341,source1_32803341_123,"Thank you, Hillary. Now women know retreat is ...",http://www.theglobeandmail.com/opinion/thank-y...,http://www.sfu.ca/content/dam/sfu/discourse-la...,And now that we've heard Heneins views as a wo...,no,1.0000,1,1,1,1.0,\n\n,no,1.0,NaN
1041,32803341,source1_32803341_124,"Thank you, Hillary. Now women know retreat is ...",http://www.theglobeandmail.com/opinion/thank-y...,http://www.sfu.ca/content/dam/sfu/discourse-la...,I will re-read this in the months or years to ...,no,1.0000,1,1,1,1.0,\n\n,no,1.0,NaN


In [17]:
train = pd.read_csv(PATH_DATA + "2829_train.csv")
test = pd.read_csv(PATH_DATA + "2829_test.csv")


X, y, y_bin = train.drop(['tag', 'bin_tag'], axis=1), train['tag'], train['bin_tag']
X_test, y_test, y_bin_test = test.drop(['tag', 'bin_tag'], axis=1), test['tag'], test['bin_tag']

X_train, X_val, y_train, y_val = train_test_split(X, y_bin, test_size=0.1, random_state=42)

... with only one file.

In [ ]:
data = pd.read_csv(PATH_DATA + "2840_full_data.csv")

# Prevent missing values
data = data.dropna(subset=['tag'])

X, y, y_bin = data.drop(['tag', 'bin_tag'], axis=1), data['tag'], data['bin_tag']

X_train, X_test, y_train, y_test =  [np.asarray(x) for x in train_test_split(X, y_bin, test_size=0.25, random_state=42)]
X_train, X_val, y_train, y_val = [np.asarray(x) for x in train_test_split(X_train, y_train, test_size=0.1, random_state=42)]

In [18]:
X_train.head()

,text_review,text_pos,lemmas,num_tokens
422,canon has kept the price of this little gem re...,NN VBZ VBN DT NN IN DT JJ NN JJ FW VBD PRP IN ...,canon have keep the price of this little gem r...,276
761,there is no static noise compared to delta's a...,EX VBZ DT JJ NN VBN TO NN POS CC DT NN VBZ JJ ...,there be no static noise compare to delta 's a...,90
2076,great deal they were beautiful and classy,JJ NN PRP VBD JJ CC JJ,great deal they be beautiful and classy,7
1902,the game is fun i would like a little more act...,DT NN VBZ NN FW MD VB DT RB RBR NN CC DT IN DT...,the game be fun i would like a little more act...,20
1740,very cute and sturdy vanity to keep all the pl...,RB JJ CC JJ NN TO VB PDT DT NN NN RB CD JJ MD ...,very cute and sturdy vanity to keep all the pl...,23
